In [1]:
from ibmm import EyeClassifier
from ibmm_online import EyeClassifierOnline
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import glob
import cv2
import os
from matplotlib import cm
from PIL import Image
from pathlib import Path
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
%matplotlib notebook

In [2]:
# Set start and end frame
start_frame = 600
end_frame = 700
frame_step = 5

# Read in ReticlePos
data = []
with open("tab55/tab55.txt","r") as f:
    next(f)
    data = f.readlines()

In [9]:
# Parse Data
x_vals = np.zeros(len(data))
y_vals = np.zeros(len(data))
frame_vals = np.zeros(len(data))

for i, line in enumerate(data):
    FrameSequence = re.findall(r"FrameSequence:\{\d+\}",line)
    ReticlePos = re.findall(r"ReticlePos:\{X\=[-]?\d+[.]?\d*[e]?[-]?\d* Y\=[-]?\d+[.]?\d*[e]?[-]?\d*\}",line)
    FrameSequence_vals = re.findall(r"\d+",FrameSequence[0])
    ReticlePos_vals = re.findall(r"[-]?\d+[.]?\d*[e]?[-]?\d*", ReticlePos[0])
    frame_vals[i] = int(FrameSequence_vals[0])
    x_vals[i] = float(ReticlePos_vals[0])
    y_vals[i] = float(ReticlePos_vals[1])


In [11]:
# Read in image files
# Take in frame number and make animation of [-5,5] second interval around specified frame
folder = "tab55"

frames = []
images = []

for image in os.listdir(folder):
    if (image.endswith(".jpg")):
        images.append(image)

images.sort()

# preallocate video array
# video = np.zeros()
print(images)

for i,image in enumerate(images[start_frame:end_frame]):
    frames.append(cv2.imread(os.path.join(folder, image)))

['tick00000.jpg', 'tick00001.jpg', 'tick00002.jpg', 'tick00003.jpg', 'tick00004.jpg', 'tick00005.jpg', 'tick00006.jpg', 'tick00007.jpg', 'tick00008.jpg', 'tick00009.jpg', 'tick00010.jpg', 'tick00011.jpg', 'tick00012.jpg', 'tick00013.jpg', 'tick00014.jpg', 'tick00015.jpg', 'tick00016.jpg', 'tick00017.jpg', 'tick00018.jpg', 'tick00019.jpg', 'tick00020.jpg', 'tick00021.jpg', 'tick00022.jpg', 'tick00023.jpg', 'tick00024.jpg', 'tick00025.jpg', 'tick00026.jpg', 'tick00027.jpg', 'tick00028.jpg', 'tick00029.jpg', 'tick00030.jpg', 'tick00031.jpg', 'tick00032.jpg', 'tick00033.jpg', 'tick00034.jpg', 'tick00035.jpg', 'tick00036.jpg', 'tick00037.jpg', 'tick00038.jpg', 'tick00039.jpg', 'tick00040.jpg', 'tick00041.jpg', 'tick00042.jpg', 'tick00043.jpg', 'tick00044.jpg', 'tick00045.jpg', 'tick00046.jpg', 'tick00047.jpg', 'tick00048.jpg', 'tick00049.jpg', 'tick00050.jpg', 'tick00051.jpg', 'tick00052.jpg', 'tick00053.jpg', 'tick00054.jpg', 'tick00055.jpg', 'tick00056.jpg', 'tick00057.jpg', 'tick00058.jp

In [12]:
video = np.array(frames, dtype="float")

In [13]:
%matplotlib inline
# np array with shape (frames, height, width, channels)

fig, ax = plt.subplots()

# Since the text file starts at frame 2, need to shift by 2
# Note: Cannot start with start_frame < 2
x = x_vals[start_frame-2:end_frame-2]
y = y_vals[start_frame-2:end_frame-2]
frames = frame_vals[start_frame-2: end_frame-2]

sc = ax.scatter(0,0,s=50)
framenum = ax.annotate(0, xy=(1000, 50))

im = plt.imshow(video[0]/np.amax(video[0]))

plt.close() # this is required to not display the generated image

def init():
    # Change BGR to RGB
    im2 = video[0].copy()/np.amax(video[0])
    im2[:, :, 0] = video[0][:, :, 2]/np.amax(video[0])
    im2[:, :, 2] = video[0][:, :, 0]/np.amax(video[0])
    im.set_data(im2)

def animate(i):
    # Change BGR to RGB
    
    im2 = video[i].copy()/np.amax(video[i])
    im2[:, :, 0] = video[i][:, :, 2]/np.amax(video[i])
    im2[:, :, 2] = video[i][:, :, 0]/np.amax(video[i])
    im.set_data(im2)
    
    # Setting reticle position
    sc.set_offsets(np.c_[x[i],y[i]])
    sc.set_color("orange")
    
    # Setting frame number
    framenum.set_text("Frame: "+str(int(frames[i])))
    
# Interval is in milliseconds
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0],
                               interval=100)
HTML(anim.to_html5_video())